### Fonctions utilitaires

Compléter en ajoutant les fonctions de padding et de découpage en bloc.

In [166]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt
import tkinter as tk
from PIL import Image, ImageTk

def load(filename):
    toLoad= Image.open(filename)
    return np.array(toLoad)


def psnr(original, compressed):
    mse = np.mean((original.astype(int) - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


### Utilitaires pour le calcul des palettes et des patchs

Fonctions auxiliaires pour manipuler les palettes, les pixels et les blocs. 

In [167]:
def load(filename):
    toLoad= Image.open(filename)
    return np.array(toLoad)

def save(matPix, filename):
    Image.fromarray(matPix).save(filename)

def padding(filename):
    mat = load(filename)

    ligne_mat = mat.shape[0]
    colonne_mat = mat.shape[1]

    #Obtenir le nombre de ligne et de colonne à rajouter.
    ligne_mat_manquante = (4 - ligne_mat % 4) % 4  
    colonne_mat_manquante = (4 - colonne_mat % 4) % 4

    #Création d'une matrice vide de la taille souhaitée. 
    mat_padding = np.zeros((ligne_mat + ligne_mat_manquante, colonne_mat + colonne_mat_manquante, 3),dtype = mat.dtype)
    mat_padding[:ligne_mat, :colonne_mat] = mat

    return mat_padding 

def decoupe_matrice(mat):
    ligne_mat = mat.shape[0]
    colonne_mat = mat.shape[1]

    #Création d'une liste qui va contenir tout les carre de 4x4 pixel en RGB.
    global Liste_des_blocs
    Liste_des_blocs=[]
    for ligne in range(0, ligne_mat, 4):
        for colonne in range(0, colonne_mat, 4):
            bloc = mat[ligne:ligne+4, colonne:colonne+4]
            Liste_des_blocs.append(bloc)

    return Liste_des_blocs

def non_padding(filename, ligne_de_base, colonne_de_base):
    #Padding
    mat_padding = padding(filename)
    #Découpe de la matrice
    Liste_des_blocs = decoupe_matrice(mat_padding)

    #Dimensions de la matrice recollée.
    ligne_mat = mat_padding.shape[0]
    colonne_mat = mat_padding.shape[1]

    #Création de la matrice recollée.
    mat_recoller = np.zeros((ligne_mat, colonne_mat, 3), dtype=mat_padding.dtype)
    for i in range(len(Liste_des_blocs)):
        indice_ligne = (i // (colonne_de_base // 4)) * 4
        indice_colonne = (i % (colonne_de_base // 4)) * 4
        mat_recoller[indice_ligne:indice_ligne+4, indice_colonne:indice_colonne+4] = Liste_des_blocs[i]

    #Retour à la matrice initiale.
    mat_de_base = mat_recoller[:ligne_de_base, :colonne_de_base]
    
    return mat_de_base

In [168]:
def tronque(nombre, p):
    #On met le nombres n en binaire.
    nombre_binaire = []
    while nombre != 0:
        nombre_binaire.append(nombre % 2)
        nombre = nombre // 2
    nombre_binaire.reverse()

    #On enlève les p bits de poids faible.
    nombre_binaire = nombre_binaire[:-p]
    
    #On remet le nombre en bianire.
    nombre_decimal = 0
    for i in range(len(nombre_binaire)):
        nombre_decimal += nombre_binaire[i] * (2 ** (len(nombre_binaire) - 1 - i))

    return nombre_decimal

def palette(a, b):
    #On crée notre palette.
    palette = np.zeros((4, 3), dtype=int)
    palette[0] = a
    palette[1] = np.round(2 * np.array(a) / 3 + np.array(b) / 3)
    palette[2] = np.round(np.array(a) / 3 + 2 * np.array(b) / 3)
    palette[3] = b
    
    return palette

def indice_couleur_la_plus_proche(palette, pixel):
    #Initialise l'indice de la couleur la plus proche avec la première couleur de la palette.
    indice_de_la_couleur_la_plus_proche = 0
    distance_euclidienne_la_plus_petite = np.linalg.norm(palette[0].astype(int) - pixel)
    
    #On test avec les autre couleur de la palette.
    for indice in range (len(palette)):
        distance_euclidienne = np.linalg.norm(palette[indice].astype(int) - pixel)
        if distance_euclidienne < distance_euclidienne_la_plus_petite:
            indice_de_la_couleur_la_plus_proche = indice

    return indice_de_la_couleur_la_plus_proche

### Les méthodes de choix des couleurs de la palette


On peut essayer plusieurs stratégies pour calculer les deux couleurs a et b:
* Calculer la couleur minimale et maximale
* Prendre des couleurs moyennes (calculer moyenne et variance pour trouver les meilleurs valeurs) 
* Calculer 10 valeurs au hasard et prendre la meilleure
* Partir d'un choix et perturber pour améliorer le résultat: on teste la meilleure modification de 16 d'un des canaux d'une des couleurs

In [169]:
def creer_couleurs_minimal_maximal(bloc_image):
    #Initialise nos canaux RGB minimal et maximal.
    rouge_minimal, rouge_maximal = 255, 0
    vert_minimal, vert_maximal = 255, 0
    bleu_minimal, bleu_maximal = 255, 0

    #On modifie nos canaux RGB minimal et maximal.
    for ligne in bloc_image:
        for colonne in ligne:
            rouge_minimal, rouge_maximal = min(rouge_minimal, colonne[0]), max(rouge_maximal, colonne[0]) 
            vert_minimal, vert_maximal = min(vert_minimal, colonne[1]), max(vert_maximal, colonne[1])    
            bleu_minimal, bleu_maximal = min(bleu_minimal, colonne[2]), max(bleu_maximal, colonne[2]) 
    
    #On met nos couleur au bon format.
    couleur_minimale = ((tronque(rouge_minimal,3)), (tronque(vert_minimal,2)), (tronque(bleu_minimal,3)))
    couleur_maximale = ((tronque(rouge_maximal,3)), (tronque(vert_maximal,2)), (tronque(bleu_maximal,3)))

    return couleur_minimale, couleur_maximale

In [170]:

def creer_couleurs_moyenne_ecart_type(bloc):
    bloc = np.array(bloc)
    moyenne_rouge = np.mean(bloc[:, :, 0])  #Moyenne des composantes rouges de tous les pixels
    ecart_type_rouge = np.std(bloc[:, :, 0])  #Écart-type des composantes rouges de tous les pixels
    moyenne_vert = np.mean(bloc[:, :, 1])  
    ecart_type_vert = np.std(bloc[:, :, 1])
    moyenne_bleu = np.mean(bloc[:, :, 2])  
    ecart_type_bleu = np.std(bloc[:, :, 2])  

    rouge_a, rouge_b = moyenne_rouge - ecart_type_rouge, moyenne_rouge + ecart_type_rouge
    vert_a, vert_b = moyenne_vert - ecart_type_vert, moyenne_vert + ecart_type_vert
    bleu_a, bleu_b = moyenne_bleu - ecart_type_bleu, moyenne_bleu + ecart_type_bleu

    couleur_a = (int(tronque(np.round(rouge_a),3)), int(tronque(np.round(vert_a),2)), int(tronque(np.round(bleu_a),3)))
    couleur_b = (int(tronque(np.round(rouge_b),3)), int(tronque(np.round(vert_b),2)), int(tronque(np.round(bleu_b),3)))

    return couleur_a, couleur_b

### Encodage d'un bloc

Écrire ici le code qui permet d'encoder un bloc et une palette en un entier.

In [171]:
def patch(tableaux_des_indices, a, b, i):
    patch_binaire = ""
    #On met en binaire l'indice de la couleur la plus proche de la palette par rapport a tout les pixels.
    for indices in tableaux_des_indices:
        pixel = Liste_des_blocs[i][indices[0], indices[1]] #Pixel du block "i" de 4x4 pixel
        indice_couleur_plus_proche = indice_couleur_la_plus_proche(palette(a, b), pixel)
        indice_couleur_plus_proche_binaire = bin(indice_couleur_plus_proche)[2:].zfill(2)
        patch_binaire += str(indice_couleur_plus_proche_binaire)

    #On ajoute nos couleur.
    patch_binaire += str(bin(b[2]))[2:].zfill(5) + str(bin(b[1]))[2:].zfill(6) + str(bin(b[0]))[2:].zfill(5) \
        + str(bin(a[2]))[2:].zfill(5) + str(bin(a[1]))[2:].zfill(6) + str(bin(a[0]))[2:].zfill(5)
    
    patch_entier = int(patch_binaire, 2)
    
    return patch_entier

### Écriture et lecture dans un fichier

In [172]:
def informations_image(type_fichier, hauteur, largeur):
    with open("informations_image.txt", "w") as f:
        f.write(type_fichier + "\n")
        f.write(str(hauteur) + " " + str(largeur) + "\n")

def ecrire_patches(tableaux_des_indices):
    with open("informations_image.txt", "a") as f:
        #Boucle pour chaque bloc
        for i in range(len(Liste_des_blocs)):
            a, b = creer_couleurs_minimal_maximal(Liste_des_blocs[i])
            #Transforme notre patch binaire en decimal.
            patch_i = str(patch(tableaux_des_indices, a, b, i))
            f.write(patch_i + "\n")

In [173]:
def Liste_des_patchs(nom_du_ficher):
    with open(nom_du_ficher, "r") as fichier:
        Liste_des_patchs  = []
        compteur = 0
        for ligne in fichier:
            if compteur < 2:
                compteur += 1
            else:
                Liste_des_patchs.append(int(ligne.strip()))
                
    return Liste_des_patchs

In [174]:
def Liste_des_patchs_en_binaire(nom_du_fichier):
    with open(nom_du_fichier, "r") as fichier:
        Liste_des_patchs_en_binaire = []
        compteur = 0
        for ligne in fichier:
            if compteur < 2:
                compteur += 1
            else:
                Liste_des_patchs_en_binaire.append(patch_binaire(int(ligne.strip())))
                
    return Liste_des_patchs_en_binaire

#Convertir un patch en binaire
def patch_binaire(patch):
    patch_binaire = bin(patch)[2:].zfill(64)  
    return patch_binaire

#Extraire la couleur à partir de la représentation binaire d'un patch
def extraire_couleur(binaire):
    couleur_a = [int(binaire[-5:]+"000", 2) , int(binaire[-11:-6]+"000", 2), int(binaire[-16:-11]+"000", 2)]
    couleur_b = [int(binaire[-21:-16]+"000", 2), int(binaire[-27:-22]+"000", 2), int(binaire[-32:-27]+"000", 2)]
    
    taille_du_tableaux_des_indices = (len(binaire[:-32])) // 2
    if taille_du_tableaux_des_indices == 16:
        tableaux_des_indices = [[3, 3], [3, 2], [3, 1], [3, 0], [2, 3], [2, 2], [2, 1], [2, 0], [1, 3], [1, 2], [1, 1], [1, 0], [0, 3], [0, 2], [0, 1], [0, 0]]
    
    return couleur_a, couleur_b, tableaux_des_indices

#Reconstruire un bloc de pixels à partir d'un patch binaire et des couleurs extraites
def reconstruire_le_bloc_de_pixel(patch_binaire, couleur_a, couleur_b, tableaux_des_indices):
    palette_bloc = palette(couleur_a, couleur_b)
    bloc = []

    for i in range(0, 32, 2):
        indice = int(patch_binaire[i:i+2], 2)
        couleur = palette_bloc[indice]
        pixel = couleur.tolist() #Convertir le pixel en liste Python
        bloc.append(pixel)

    if tableaux_des_indices == [[3, 3], [3, 2], [3, 1], [3, 0], [2, 3], [2, 2], [2, 1], [2, 0], [1, 3], [1, 2], [1, 1], [1, 0], [0, 3], [0, 2], [0, 1], [0, 0]]:
        bloc_pixels = [[[],[],[],[]],[[],[],[],[]],[[],[],[],[]],[[],[],[],[]]]

    k = 0
    for i in range(4):
        for j in range(4):
            bloc_pixels[i][j] = bloc[k]
            k += 1

    return bloc_pixels

#Reconstruire l'image à partir des blocs de pixels
def reconstruire_le_image(nom_du_fichier, ligne_actuelle, colonne_actuelle, ligne_de_base, colonne_de_base):
    #Création des blocs
    Liste_des_patchs = Liste_des_patchs_en_binaire(nom_du_fichier)
    Liste_des_blocs = []
    for i in range(len(Liste_des_patchs)):
        couleur_a, couleur_b, tableaux_des_indices = extraire_couleur(Liste_des_patchs[i])
        bloc_pixels = reconstruire_le_bloc_de_pixel(Liste_des_patchs[i], couleur_a, couleur_b, tableaux_des_indices)
        Liste_des_blocs.append(bloc_pixels)

    #Création de l'image recollée
    image_recoller = np.zeros((ligne_actuelle, colonne_actuelle, 3), dtype=np.uint8)
    for i in range(len(Liste_des_blocs)):
        indice_ligne = (i // (colonne_de_base // 4)) * 4
        indice_colonne = (i % (colonne_de_base // 4)) * 4
        image_recoller[indice_ligne:indice_ligne+4, indice_colonne:indice_colonne+4] = Liste_des_blocs[i]

    #Retour à l'image initiale
    image_de_base = image_recoller[:ligne_de_base, :colonne_de_base]

    return image_de_base

### Fonctions de test

Écrire ici tous vos tests. Chaque fonction écrite doit être testée.

In [175]:
#Test de la fonction "padding"

mat_padding = padding("proc.jpg")
save(mat_padding, "proc_padding.jpg")

In [176]:
#Test de la fonction "decoupe_matrice"

decoupe_matrice(mat_padding)
bloc = Liste_des_blocs[0]
print(bloc)

[[[94 54 18]
  [94 54 18]
  [94 54 18]
  [94 54 18]]

 [[94 54 18]
  [94 54 18]
  [94 54 18]
  [94 54 18]]

 [[94 54 18]
  [94 54 18]
  [94 54 18]
  [94 54 18]]

 [[95 55 19]
  [95 55 19]
  [95 55 19]
  [95 55 19]]]


In [177]:
#Test de la fonction "non_padding"

mat_de_base = non_padding("proc.jpg",1537,2048)
save(mat_de_base, "proc_non_padding.jpg")

In [178]:
#Test de la fonction "tronque"

n = 255
p = 3   
n_tronque = tronque(n, p)
print(n_tronque)

31


In [179]:
#Test de la fonction "palette"

a = [tronque(90,3),tronque(90,2),tronque(90,3)]
b = [tronque(180,3),tronque(180,2),tronque(180,3)]
c = palette(a, b)
print(c)

[[11 22 11]
 [15 30 15]
 [18 37 18]
 [22 45 22]]


In [180]:
#Test de la fonction "indice_de_la_couleur_la_plus_proche"

pixel = [100, 100, 100]
indice_de_la_couleur_la_plus_proche = indice_couleur_la_plus_proche(c, pixel)
print(indice_de_la_couleur_la_plus_proche)

3


In [181]:
#Test de la fonction "creer_couleurs_minimal_maximal"

couleur_minimal, couleur_maximal = creer_couleurs_minimal_maximal(Liste_des_blocs[0])
print(couleur_minimal, couleur_maximal)

(11, 13, 2) (11, 13, 2)


In [182]:
#Test de la fonction "creer_couleurs_moyenne_ecart_type"

couleur_a, couleur_b = creer_couleurs_moyenne_ecart_type(Liste_des_blocs[0])
print(couleur_a, couleur_b)

(11, 13, 2) (11, 13, 2)


In [183]:
#Test de la fonction "patch"

tableaux_des_indices = [[3,3],[3,2],[3,1],[3,0],[2,3],[2,2],[2,1],[2,0],[1,3],[1,2],[1,1],[1,0],[0,3],[0,2],[0,1],[0,0]]
a, b = creer_couleurs_minimal_maximal(Liste_des_blocs[0])
patch_0 = patch(tableaux_des_indices, a, b, 0)
print(patch_0)

296423851


In [184]:
#Test de la fonction "informations_image"

informations_image("BC1", 1537, 2048)

In [185]:
#Test de la fonction "ecrire_patches"

tableaux_des_indices = [[3,3],[3,2],[3,1],[3,0],[2,3],[2,2],[2,1],[2,0],[1,3],[1,2],[1,1],[1,0],[0,3],[0,2],[0,1],[0,0]]
ecrire_patches(tableaux_des_indices)

In [186]:
#Test de la fonction "Liste_des_patch"

Liste_des_patchs("informations_image.txt")

[296423851,
 18446744069713105323,
 298521035,
 298521035,
 298521035,
 298521035,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 300618219,
 18446744069715268075,
 300683756,
 300618219,
 300618219,
 18446744069715202507,
 298521035,
 298521035,
 298521035,
 18446744069847323083,
 18446744069983640011,
 18446744070122054123,
 18446744070396848683,
 18446744070669482603,
 18446744070807964332,
 18446744071078501100,
 18446744071216980781,
 18446744071221177198,
 18446744071357557646,
 18446744071493876687,
 18446744071630257135,
 18446744071632356368,
 2217772080,
 18446744071770836048,
 2356251761,
 18446744071907150961,
 18446744071907218577,
 2492634258,
 18446744072043533458,
 2628951218,
 2628951218,
 18446744072045630642,
 18446744072045630642,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631048402,
 2631113939,
 263111393

In [187]:
#Test des fonctions pour recrée une image a partir de son fichier BC1

patch_0 = 296423851
#Test de la fonction qui transforme notre patch en binaire.
patch_0_binaire = patch_binaire(patch_0)
print(patch_0_binaire)
#Test de la fonction qui extraite les couleur et le tableaux des indices.   
couleur_a, couleur_b, tableaux_des_indices = extraire_couleur(patch_0_binaire)
print(couleur_a, couleur_b, tableaux_des_indices)
#Test de la fonction qui recontrui notre bloc de pixel.
bloc_pixelx = reconstruire_le_bloc_de_pixel(patch_0_binaire, couleur_a, couleur_b, tableaux_des_indices)
print(bloc_pixelx)

#Test de la fonction qui recrée notre image.
image = reconstruire_le_image("informations_image.txt", 1540, 2048, 1537, 2048)
save(image, "image_reconstruite.jpg")

0000000000000000000000000000000000010001101010110001000110101011
[88, 48, 16] [88, 48, 16] [[3, 3], [3, 2], [3, 1], [3, 0], [2, 3], [2, 2], [2, 1], [2, 0], [1, 3], [1, 2], [1, 1], [1, 0], [0, 3], [0, 2], [0, 1], [0, 0]]
[[[88, 48, 16], [88, 48, 16], [88, 48, 16], [88, 48, 16]], [[88, 48, 16], [88, 48, 16], [88, 48, 16], [88, 48, 16]], [[88, 48, 16], [88, 48, 16], [88, 48, 16], [88, 48, 16]], [[88, 48, 16], [88, 48, 16], [88, 48, 16], [88, 48, 16]]]


In [190]:
# Fonction pour charger et afficher une image
def afficher_image(nom_du_fichier):
    image = Image.open(nom_du_fichier)
    image = image.resize((450, 350))
    photo = ImageTk.PhotoImage(image)

    label_image = tk.Label(fenetre, image=photo)
    label_image.image = photo
    label_image.grid(row=6, column=0, columnspan=3, padx=10, pady=10)

    label_image.after(30000, label_image.destroy)

#Fonctions pour les actions des boutons
def Test_de_la_fonction_padding():
    mat_padding = padding("proc.jpg")
    save(mat_padding, "proc_padding.jpg")
    label_resultat.config(text="Le padding ajoute des pixels noirs pour que l'image ait une hauteur et une largeur divisibles par 4 si nécessaire.")
    afficher_image("proc_padding.jpg")

def Test_de_la_fonction_decoupe_matrice():
    decoupe_matrice(mat_padding)
    bloc = Liste_des_blocs[0]
    label_resultat.config(text="Le premier bloc de 4x4 pixels est : \n" + str(bloc))

def Test_de_la_fonction_non_padding():
    mat_de_base = non_padding("proc.jpg", 1537, 2048)
    save(mat_de_base, "proc_non_padding.jpg")
    label_resultat.config(text="Le non-padding supprime les pixels noirs autour de l'image.")
    afficher_image("proc_non_padding.jpg")

def Test_de_la_fonction_tronque():
    n = 255
    p = 3
    n_tronque = tronque(n, p)
    label_resultat.config(text="Le tronquage de 255 en enlevant les 3 derniers bits donne : " + str(n_tronque))

def Test_de_la_fonction_palette():
    a = [tronque(90,3), tronque(90,2), tronque(90,3)]
    b = [tronque(180,3), tronque(180,2), tronque(180,3)]
    c = palette(a, b)
    label_resultat.config(text="Si on prend les couleurs a [90,90,90] et b [180,180,180] mis ensuite au bon format, la palette est :\n" + str(c))

def Test_de_la_fonction_couleur_la_plus_proche():
    pixel = [100, 100, 100]
    indice_couleur_la_plus_proche = indice_couleur_la_plus_proche(c, pixel)
    label_resultat.config(text="L'indice de la couleur la plus proche dans la palette pour le pixel [100, 100, 100] est : " +str(indice_couleur_la_plus_proche))

def Test_de_la_fonction_creer_couleurs_min_max():
    couleur_minimal, couleur_maximal = creer_couleurs_minimal_maximal(Liste_des_blocs[0])
    label_resultat.config(text="Pour le premier bloc de 4x4 pixels de l'image, la couleur minimale est : " + str(couleur_minimal) + "la couleur maximale est : " + str(couleur_maximal))

def Test_de_la_fonction_creer_couleurs_moyenne_ecart_type():
    couleur_a, couleur_b = creer_couleurs_moyenne_ecart_type(Liste_des_blocs[0])
    label_resultat.config(text="Pour le premier bloc de 4x4 pixels de l'image, la couleur minimale est : " + str(couleur_a) + "la couleur maximale est : " + str(couleur_b))

def Test_de_la_fonction_patch():
    tableaux_des_indices = [[3,3],[3,2],[3,1],[3,0],[2,3],[2,2],[2,1],[2,0],[1,3],[1,2],[1,1],[1,0],[0,3],[0,2],[0,1],[0,0]]
    a, b = creer_couleurs_minimal_maximal(Liste_des_blocs[0])
    patch_0 = patch(tableaux_des_indices, a, b, 0)
    label_resultat.config(text=("Le patch du premier bloc de l'image avec la méthode des couleurs minimales et maximales est :\n" + str(patch_0)))

def Test_de_la_fonction_informations_image():
    label_resultat.config(text="Ouvrir le fichier informations_image.txt")

def Test_de_la_fonction_ecrire_patches():
    label_resultat.config(text="Ouvrir le fichier informations_image.txt")
    
def Test_de_la_fonction_Liste_des_patch():
    aladin = Liste_des_patchs("informations_image.txt")[:3] 
    label_resultat.config(text="Liste des 3 premières valeurs du patch : " + str(aladin) + ". Pour la suite, ouvrez le fichier informations_image.txt")

def Test_de_la_reconstrution_des_images():
    label_resultat.config(text="On recrée une image a partir de notre fichier informations_image.txt.")
    afficher_image("image_reconstruite.jpg")
    
#Fenêtre principale
fenetre = tk.Tk()
fenetre.title("Compression BC1")  
fenetre.configure(background="beige")  

#Créer les boutons avec leur fonction
Button_fonction_padding = tk.Button(fenetre, text="Padding", font=("Cambria", 11), command=Test_de_la_fonction_padding)
Button_fonction_decoupe_matrice = tk.Button(fenetre, text="Découpe matrice", font=("Cambria", 11), command=Test_de_la_fonction_decoupe_matrice)
Button_fonction_non_padding = tk.Button(fenetre, text="Non padding", font=("Cambria", 11), command=Test_de_la_fonction_non_padding)
Button_fonction_tronque = tk.Button(fenetre, text="Tronque", font=("Cambria", 11), command=Test_de_la_fonction_tronque)
Button_fonction_palette = tk.Button(fenetre, text="Palette", font=("Cambria", 11), command=Test_de_la_fonction_palette)
Button_fonction_couleur_la_plus_proche = tk.Button(fenetre, text="Indice de la couleur la plus proche", font=("Cambria", 11), command=Test_de_la_fonction_couleur_la_plus_proche)
Button_fonction_creer_couleurs_min_max = tk.Button(fenetre, text="Couleurs minimal maximal", font=("Cambria", 11), command=Test_de_la_fonction_creer_couleurs_min_max)
Button_fonction_creer_couleurs_moyenne_ecart_type = tk.Button(fenetre, text="Couleurs moyenne ecart type", font=("Cambria", 11), command=Test_de_la_fonction_creer_couleurs_moyenne_ecart_type)
Button_fonction_patch = tk.Button(fenetre, text="Patch", font=("Cambria", 11), command=Test_de_la_fonction_patch)
Button_fonction_informations_image = tk.Button(fenetre, text="Informations image", font=("Cambria", 11), command=Test_de_la_fonction_informations_image)
Button_fonction_ecrire_patches = tk.Button(fenetre, text="Ecrire patches", font=("Cambria", 11), command=Test_de_la_fonction_ecrire_patches)
Button_fonction_Liste_des_patch = tk.Button(fenetre, text="Liste des patches", font=("Cambria", 11), command=Test_de_la_fonction_Liste_des_patch)
Button_fonction_reconstrution_des_images = tk.Button(fenetre, text="Reconstrution des images", font=("Cambria", 11), command=Test_de_la_reconstrution_des_images)

#Positionner les boutons dans la grille
Button_fonction_padding.grid(row=0, column=0, padx=10, pady=10)
Button_fonction_decoupe_matrice.grid(row=0, column=1, padx=10, pady=10)
Button_fonction_non_padding.grid(row=0, column=2, padx=10, pady=10)
Button_fonction_tronque.grid(row=1, column=0, padx=10, pady=10)
Button_fonction_palette.grid(row=1, column=1, padx=10, pady=10)
Button_fonction_couleur_la_plus_proche.grid(row=1, column=2, padx=10, pady=10)
Button_fonction_creer_couleurs_min_max.grid(row=2, column=0, padx=10, pady=10)
Button_fonction_creer_couleurs_moyenne_ecart_type.grid(row=2, column=1, padx=10, pady=10)
Button_fonction_patch.grid(row=2, column=2, padx=10, pady=10)
Button_fonction_informations_image.grid(row=3, column=0, padx=10, pady=10)
Button_fonction_ecrire_patches.grid(row=3, column=1, padx=10, pady=10)
Button_fonction_Liste_des_patch.grid(row=3, column=2, padx=10, pady=10)
Button_fonction_reconstrution_des_images.grid(row=4, column=1, padx=10, pady=10)

#Afficher le résultat
label_resultat = tk.Label(fenetre, text="", font=("Cambria", 11), bg="white", fg="black", padx=10, pady=10, relief=tk.RIDGE, borderwidth=2)
label_resultat.grid(row=5, column=0, columnspan=3, padx=10, pady=10, sticky="nsew")

#Lancer la boucle principale
fenetre.mainloop()